# 問題1 学習と推定

Faster_R-CNN.ipynb参照

# 問題2 論文と実装の対応

重要だと思う部分のコードに関して、#の青文字で記載

In [1]:
from keras import backend as K
from keras.optimizers import Adam
from keras.layers import Input
from keras.models import Model

from model import losses as losses
from model import resnet as nn

#おそらくCがRPNの何か
def get_model(C, classes_count):
    #画像の入力
    img_input = Input(shape=(None, None, 3))
    #region_proposalの入力
    roi_input = Input(shape=(C.num_rois, 4))

    # define the base network (resnet here)
    # RPNとFast R-CNNで共有する特徴マップ
    shared_layers = nn.nn_base(img_input, trainable=True)

    # define the RPN, built on the base layers
    #アンカーボックス数を設定
    num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
    #アンカーの数、特徴マップを渡してrpnを生成
    #特徴マップに対してconv2dを適用して各アンカーボックスに対する奥行き（オフセット、クラス）を生成する
    rpn = nn.rpn(shared_layers, num_anchors)

    # define the classifer, built on the base layers
    #引数=(resnetによって抽出された特徴マップ、roiのインプット、roiのインプット数、クラス数、trainable)
    classifier = nn.classifier(shared_layers, roi_input, C.num_rois, nb_classes=len(classes_count), trainable=True)

    # defining the models and a model that holds both the RPN and the classifier
    model_rpn = Model(img_input, rpn[:2])
    model_classifier = Model([img_input, roi_input], classifier)
    model_all = Model([img_input, roi_input], rpn[:2] + classifier)

    model_rpn.compile(optimizer=Adam(lr=1e-4), loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
    model_classifier.compile(optimizer=Adam(lr=1e-4), loss=[losses.class_loss_cls, losses.class_loss_regr(len(classes_count)-1)], metrics={'dense_class_{}'.format(len(classes_count)): 'accuracy'})
    model_all.compile(optimizer='sgd', loss='mae')

    return model_rpn, model_classifier, model_all

Using TensorFlow backend.


ImportError: cannot import name 'losses'

# resnet

In [3]:
#__future__は言語に関数基本的な事柄を変更する

from __future__ import print_function
#相対インポートの設定
from __future__ import absolute_import 

from keras.layers import Input, Add, Dense, Activation, Flatten,\
Convolution2D, AveragePooling2D,TimeDistributed

from keras import backend as K



def nn_base(input_tensor=None, trainable=False):
    
    #resnetへの入力の形状(不明のとき)
    input_shape = (None, None, 3)
    
    if input_tensor is None:
        #input_tensorが空の場合ここでkerasのinputを生成
        img_input = Input(shape=input_shape)
    else:
        #input_tensorが空出ないが、kerasのテンソルでない場合
        if not K.is_keras_tensor(input_tensor):
            #keras化する
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            #keras型なのでそのまま入れる
            img_input = input_tensor
    
    bn_axis = 3#channel数
    #ZeroPadding2Dで上下にpaddingを追加する
    x = ZeroPadding2D((3, 3))(img_input)
    #trainable=Trueで訓練する
    x = Convolution2D(64,(7,7), strides=(2,2), name='conv1',
                      trainble=trainable)
    x = FixedBatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    
            
            

In [1]:
#FixedBatchNormalization
from keras.engine import Layer, InputSpec
from keras import initialziers, regularizers
from keras import backend as K

class FixedBatchNormalization(Layer):
    def __init__(self, epsilon=1e-3, axis=-1,
                weights=None, beta_init='zero', gamma_init='one',
                gamma_regularizer=None, beta_regularizer=None,
                **kwargs):
        
        self.supports_masking = True
        self.beta_init = initializiers.get(beta_init)
        print(self.beta_init)
        #初期化方法を定義
        self.gamma_init = initializiers.get(gamma_init)
        print(self.gamma_init)
        self.epsilon = epsilon
        self.axis = axis
        #正規化の方法を定義
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        print(self.gamma_regularizer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        print(self.beta_regularizer)
        self.initial_weights = weights
        super(FixedBatchNormalization, self).__init__(**kwargs)
        
    #初期値を定義する
    def build(self, input_shape):
#         add_weight（）を呼び出してから、スーパーのbuild（）を呼び出します
#         （self.built = Trueに設定します。これは、ユーザーが最初の__call__の
#         前にbuild（）を手動で呼び出したい場合に便利です）。
        
        #つまりbuildを呼び出す前のadd_weightの設定を行う
        #built=Trueにすることでカスタムのbuildの設定を完了する
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (input_shape[self.axis],)#(C,)
        #shape=(C,)でgamma,beta,mean,stdを定義
        self.gamma = self.add_weight(shape,
                                    initializer=self.gamma_init,
                                    regularizer=self.gamma_regularizer,
                                    name='{}_gamma'.format(self.name),
                                    trainable=False)
        self.beta = self.add_weight(shape,
                                   initializer=self.beta_init,
                                   regularizer=self.beta_regularizer,
                                   name='{}_beta'.format(self.name),
                                   trainable=False)
        self.running_mean = self.add_weight(shape,initializer='zero',
                                           name='{}_running_mean'.format(self.name))
        self.running_std = self.add_weight(shape, initializer='one',
                                          name='{}_running_std'.format(self.name))
        
        print(self.running_std)
        
        if self.initial_weights is not None:
            #引数にweightがあればweightを設定する
            self.set_weights(self.initial_weights)
            del self.initial_weights
            
        self.built = True
        
    def call(self, x, mask=None):
        #assert 条件式　エラー文
        #条件式がfalseの場合、エラー文を投げる。
        #この場合buildが完了しているか確認している
        assert self.built, 'Layer must be built before being called'
        
        input_shape = K.int_shape(x)#xのshape(tuple)を獲得
        reduction_axes = list(range(len(input_shape)))#shapeのリスト
        del reduction_axes[self.axis]#channel方向を削除
        
        
        
        
        
        
        
        
        

ModuleNotFoundError: No module named 'keras'

# 問題3 学習済みの重みによる推定

実際にbboxを付した画像は、Google colab上で行なっていたため、セッションが切れた際に紛失しました。ログが残っているのでそちらを記します。

In [2]:
Found 1 boxes for img
person 0.73 (26, 79) (554, 409)
4.272179343000062
Input image filename:/content/keras-yolo3/simpsons_dataset/carl_carlson/pic_0004.jpg
(416, 416, 3)
Found 0 boxes for img
1.7000756250004088
Input image filename:/content/keras-yolo3/simpsons_dataset/carl_carlson/pic_0001.jpg
(416, 416, 3)
Found 1 boxes for img
person 0.91 (227, 82) (320, 450)
1.6344884119998824
Input image filename:/content/keras-yolo3/simpsons_dataset/carl_carlson/pic_0085.jpg
(416, 416, 3)
Found 0 boxes for img
1.6449777230000109
Input image filename:/content/keras-yolo3/simpsons_dataset/carl_carlson/pic_0085.jpg
(416, 416, 3)
Found 0 boxes for img
1.6431265849996635
Input image filename:/content/keras-yolo3/simpsons_dataset/carl_carlson/pic_0077.jpg
(416, 416, 3)
Found 1 boxes for img
fire hydrant 0.40 (97, 28) (250, 242)
1.6575630780007486
Input image filename:/content/keras-yolo3/simpsons_dataset/barney_gumble/pic_0001.jpg
(416, 416, 3)
Found 0 boxes for img
1.6473533620001035
Input image filename:/content/keras-yolo3/simpsons_dataset/barney_gumble/pic_0005.jpg
(416, 416, 3)
Found 0 boxes for img
1.6330609899996489
Input image filename:/content/keras-yolo3/simpsons_dataset/barney_gumble/pic_0011.jpg
(416, 416, 3)
Found 0 boxes for img
1.6330044970000017
Input image filename:/content/keras-yolo3/simpsons_dataset/chief_wiggum/pic_0003.jpg
(416, 416, 3)
Found 0 boxes for img
1.6340531550004016
Input image filename:/content/keras-yolo3/simpsons_dataset/homer_simpson/pic_0002.jpg
(416, 416, 3)
Found 0 boxes for img
1.954918448999706
Input image filename:/content/keras-yolo3/simpsons_dataset/homer_simpson/pic_0005.jpg
(416, 416, 3)
Found 0 boxes for img
1.6352981849995558
Input image filename:/content/keras-yolo3/simpsons_dataset/gil/pic_0007.jpg
(416, 416, 3)
Found 1 boxes for img
kite 0.47 (107, 13) (619, 346)
1.6652927129998716
Input image filename:Traceback (most recent call last):
  File "yolo_video.py", line 72, in <module>
    detect_img(YOLO(**vars(FLAGS)))
  File "yolo_video.py", line 8, in detect_img
    img = input('Input image filename:')
KeyboardInterrupt

SyntaxError: invalid syntax (<ipython-input-2-5d298dbf273e>, line 1)

# 問題4 学習のためのファイルを作成

annotation.csv参照

# 問題5 学習が行えることの確認

学習が実行できたことを示すログを下記に添付します。

In [3]:
Using TensorFlow backend.
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:107: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:111: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:203: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-12-18 11:41:24.903793: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-12-18 11:41:24.904023: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2114bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-18 11:41:24.904057: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-18 11:41:24.906129: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-18 11:41:24.908098: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2019-12-18 11:41:24.908137: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2589ef97a2b3): /proc/driver/nvidia/version does not exist
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:216: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:223: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2041: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2239: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

Create YOLOv3 model with 9 anchors and 18 classes.
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 69) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((69,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 69) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((69,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer conv2d_75 due to mismatch in shape ((1, 1, 256, 69) vs (255, 256, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer conv2d_75 due to mismatch in shape ((69,) vs (255,)).
  weight_values[i].shape))
Load weights model_data/yolo_weights.h5.
Freeze the first 249 layers of total 252 layers.
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1702: The name tf.log is deprecated. Please use tf.math.log instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3351: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

Train on 6077 samples, val on 675 samples, with batch size 32.
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

Epoch 1/50
2019-12-18 11:41:44.705497: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:533] shape_optimizer failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)
2019-12-18 11:41:44.800962: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:533] remapper failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)
2019-12-18 11:41:45.665726: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:533] shape_optimizer failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)
2019-12-18 11:41:45.738936: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:533] remapper failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)
2019-12-18 11:41:47.321074: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 708837376 exceeds 10% of system memory.
2019-12-18 11:41:53.047770: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 712249344 exceeds 10% of system memory.
2019-12-18 11:41:55.280101: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 354418688 exceeds 10% of system memory.
tcmalloc: large alloc 1594982400 bytes == 0xd5504000 @  0x7f7fc7dc91e7 0x7f7f8df6f192 0x7f7f91b4416a 0x7f7f91b87019 0x7f7f91b891e7 0x7f7f91b899f1 0x7f7f91bcf8db 0x7f7f91bcff7c 0x7f7f91bd133c 0x7f7f893e7f36 0x7f7f893da585 0x7f7f894986b9 0x7f7f89495d88 0x7f7fc66ab66f 0x7f7fc778d6db 0x7f7fc7ac688f
2019-12-18 11:41:57.988557: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 354418688 exceeds 10% of system memory.
2019-12-18 11:42:00.784998: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 354418688 exceeds 10% of system memory.
tcmalloc: large alloc 1594982400 bytes == 0xbffc2000 @  0x7f7fc7dc91e7 0x7f7f8df6f192 0x7f7f91b4416a 0x7f7f91b87019 0x7f7f91b891e7 0x7f7f91b899f1 0x7f7f91bcf8db 0x7f7f91bcff7c 0x7f7f91bd133c 0x7f7f893e7f36 0x7f7f893da585 0x7f7f894986b9 0x7f7f89495d88 0x7f7fc66ab66f 0x7f7fc778d6db 0x7f7fc7ac688f
  1/189 [..............................] - ETA: 4:20:21 - loss: 8217.9639tcmalloc: large alloc 1594982400 bytes == 0x149856000 @  0x7f7fc7dc91e7 0x7f7f8df6f192 0x7f7f91b4416a 0x7f7f91b87019 0x7f7f91b891e7 0x7f7f91b899f1 0x7f7f91bcf8db 0x7f7f91bcff7c 0x7f7f91bd133c 0x7f7f893e7f36 0x7f7f893da585 0x7f7f894986b9 0x7f7f89495d88 0x7f7fc66ab66f 0x7f7fc778d6db 0x7f7fc7ac688f
tcmalloc: large alloc 1594982400 bytes == 0x134314000 @  0x7f7fc7dc91e7 0x7f7f8df6f192 0x7f7f91b4416a 0x7f7f91b87019 0x7f7f91b891e7 0x7f7f91b899f1 0x7f7f91bcf8db 0x7f7f91bcff7c 0x7f7f91bd133c 0x7f7f893e7f36 0x7f7f893da585 0x7f7f894986b9 0x7f7f89495d88 0x7f7fc66ab66f 0x7f7fc778d6db 0x7f7fc7ac688f
 12/189 [>.............................] - ETA: 2:37:12 - loss: 4657.3961

SyntaxError: invalid syntax (<ipython-input-3-519264450b9a>, line 1)